In [1]:
import joblib
import numpy as np

from langchain.chat_models import ChatOpenAI
from discussion_agents.cog.agent.reflexion import ReflexionCoTAgent

TypeError: model_validator() missing 1 required keyword-only argument: 'mode'

In [ ]:
hotpot = joblib.load('hotpot-qa-distractor-sample.joblib').reset_index(drop = True)

hotpot['supporting_paragraphs'] = None
for ind, row in hotpot.iterrows():
    supporting_articles = row['supporting_facts']['title']
    articles = row['context']['title']
    sentences = row['context']['sentences'] 
    supporting_paragraphs = []
    for article in supporting_articles:
        supporting_paragraph = ''.join(sentences[np.where(articles == article)][0])
        supporting_paragraphs.append(supporting_paragraph)
    supporting_paragraphs = '\n\n'.join(supporting_paragraphs)
    hotpot.at[ind, 'supporting_paragraphs'] = supporting_paragraphs

In [ ]:
row = hotpot.iloc[0]

import dotenv
import os

dotenv.load_dotenv("../../.env")
openai_api_key = os.getenv("OPENAI_API_KEY")

llm = ChatOpenAI(
    temperature=0,
    max_tokens=250,
    model_name="gpt-3.5-turbo",
    model_kwargs={"stop": "\n"},
    openai_api_key=openai_api_key
)

agent = ReflexionCoTAgent(
    self_reflect_llm=llm,
    action_llm=llm,
    question=row['question'],
    context=row['supporting_paragraphs'],
    key=row['answer']
)

In [ ]:
row["question"]

"VIVA Media AG changed it's name in 2004. What does their new acronym stand for?"

In [ ]:
row["answer"]

'Gesellschaft mit beschränkter Haftung'

In [ ]:
agent.key

'Gesellschaft mit beschränkter Haftung'

In [ ]:
agent.answer

'Company with Limited Liability'

In [ ]:
agents = [agent]
n = 5
for i in range(n):
    for agent in [a for a in agents if not a.is_correct()]:
        agent.run(reflexion_strategy=None)
        print(f'Answer: {agent.key}')

Thought: The question is asking for the acronym that VIVA Media AG changed its name to in 2004. Based on the context, I know that VIVA Media AG is now known as VIVA Media GmbH. Therefore, the acronym "GmbH" stands for "Gesellschaft mit beschränkter Haftung" in German, which translates to "company with limited liability" in English.
Action: Finish[Company with Limited Liability]
Answer: Gesellschaft mit beschränkter Haftung
Thought: The question is asking for the acronym that VIVA Media AG changed its name to in 2004. Based on the context, I know that VIVA Media AG is now known as VIVA Media GmbH. Therefore, the acronym "GmbH" stands for "Gesellschaft mit beschränkter Haftung" in German, which translates to "company with limited liability" in English.
Action: Finish[Company with Limited Liability]
Answer: Gesellschaft mit beschränkter Haftung
Thought: The question is asking for the acronym that VIVA Media AG changed its name to in 2004. Based on the context, I know that VIVA Media AG is

In [ ]:
f"\ns" == "\ns"